In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json
import re

C:\Users\fabri\AppData\Local\Temp\ipykernel_22780\3313601917.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Importer les données des capteurs TESSW

In [2]:
def concat_sensors_data(initial_path):
    """
    Concatène les données de plusieurs capteurs à partir de fichiers CSV dans les sous-dossiers.

    Args:
    - initial_path (str): Le chemin du dossier initial contenant les sous-dossiers des capteurs.
    - numberOfSensors (int): Le nombre total de capteurs à considérer.

    Returns:
    - DataFrame: Un DataFrame contenant les données concaténées de tous les capteurs.
    """
    # Créer une expression régulière pour extraire les numéros de dossier
    pattern = re.compile(r'stars(\d+)')
    
    # Initialiser une liste pour stocker les DataFrames de chaque capteur
    sensor_dataframes = []

    # Ouvrir le fichier JSON contenant les coordonnées des capteurs et le transformer en dict python
    f = open('../data/sensorCoord.json')
    sensorCoord = json.load(f)
    
    # Liste pour stocker les numéros de dossier des capteurs sélectionnés
    selected_sensor_numbers = []
    
    # Parcourir les dossiers dans le chemin initial
    for folder_name in os.listdir(initial_path):
        # Vérifier si le nom de dossier correspond au motif attendu (stars suivi d'un nombre)
        match = pattern.match(folder_name)
        if match:
            sensor_number = int(match.group(1))
            selected_sensor_numbers.append(sensor_number)
            sensor_folder = os.path.join(initial_path, folder_name)
            
            # Initialiser une liste pour stocker les DataFrames de chaque fichier
            sensor_files_dataframes = []
            
            # Parcourir les fichiers dans le dossier du capteur
            for file_name in os.listdir(sensor_folder):
                if file_name.endswith('.dat'):
                    file_path = os.path.join(sensor_folder, file_name)
                    # Lire le fichier de données et stocker son DataFrame dans la liste
                    cols = ['UTCDataTime', 'LocalDateTime', 'EnclosureTemp', 'SkyTemp', 'Frequency', 'MSAS', 'ZP']

                    # Lire le fichier de données et stocker son DataFrame dans la liste
                    df = pd.read_csv(file_path, delimiter=';', names=cols, skiprows=35)
                    df['latitude'] = sensorCoord[f"stars{sensor_number}"][0]
                    df['longitude'] = sensorCoord[f"stars{sensor_number}"][1]
                    df['sensor'] = f"stars{sensor_number}"
                    sensor_files_dataframes.append(df)
            
            # Concaténer les DataFrames de chaque fichier et les ajouter à la liste des DataFrames de capteur
            if sensor_files_dataframes:
                sensor_dataframes.append(pd.concat(sensor_files_dataframes, ignore_index=True))
    
    # Sélectionner les numéros de capteurs à traiter
    selected_sensor_numbers.sort()
    # selected_sensor_numbers = selected_sensor_numbers[:numberOfSensors]
    
    # Afficher les capteurs sélectionnés
    print("Capteurs sélectionnés :", selected_sensor_numbers)
    
    # Concaténer les DataFrames de chaque capteur en un seul DataFrame
    if sensor_dataframes:
        concatenated_data = pd.concat(sensor_dataframes)
        return concatenated_data
    else:
        print("Aucun fichier de données trouvé pour les capteurs.")
        return None



In [3]:
# Exemple d'utilisation
initial_path = "../data/data_capteurs/"
startSensorNumber = 251  # Numéro du premier capteaur
# numberOfSensors = 36  # Exemple avec 8 capteurs
df_sensors = concat_sensors_data(initial_path)
df_sensors.head()

Capteurs sélectionnés : [251, 712, 713, 737, 873, 874, 875, 876, 877, 881, 952, 954, 955, 956, 958, 959, 960, 961, 962, 963, 964, 965, 967, 969, 970, 971, 972, 974, 975, 1022, 1023, 1025, 1026, 1029, 1032, 1033]


,UTCDataTime,LocalDateTime,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,latitude,longitude,sensor
0,2023-11-02T12:57:05.000,2023-11-02T12:57:05.000,31.1,21.7,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
1,2023-11-02T12:58:05.000,2023-11-02T12:58:05.000,31.2,21.7,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
2,2023-11-02T12:59:05.000,2023-11-02T12:59:05.000,31.5,21.8,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
3,2023-11-02T13:18:05.000,2023-11-02T13:18:05.000,28.9,20.8,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
4,2023-11-02T13:19:05.000,2023-11-02T13:19:05.000,28.7,20.4,50000.0,0.0,2.0,-21.3587,55.66335,stars1022


## Importer les données météo

In [4]:
def import_meteo_data(csv_file):

    chunk_size = 10000  # Choisir une taille de morceau appropriée

    chunks = pd.read_csv(csv_file, delimiter=";", chunksize=chunk_size)

    # Liste pour stocker les morceaux lus
    chunk_list = []

    for chunk in chunks:
        # Traitement ou manipulation des données du chunk
        chunk_list.append(chunk)

    # Concaténation des morceaux
    df = pd.concat(chunk_list)

    col_list = ["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "ALTI", "AAAAMMJJHH", "RR1", "DRR1", "TD", "TN", "HTN", "TX", "HTX", "U", "UN", "HUN", "UX", "HUX", "B1", "N2", "B2", "N3", "B3"]
    
    # Sélectionner les colonnes qui nous intéressent
    new_df = df[col_list]
    return new_df


#### Fichier de données 2010 - 2019

In [5]:
data_2010_2019 = "../data/data_meteo/H_974_2010-2019.csv"
df_2010_2019 = import_meteo_data(data_2010_2019)

In [6]:
# df_2016_2019 = new_df_2010_2019.dropna(thresh=16)
# df_2016_2019
# Convertir la colonne en chaînes de caractères
df_2010_2019['AAAAMMJJHH'] = df_2010_2019['AAAAMMJJHH'].astype(str)
df_2016_2019 = df_2010_2019[df_2010_2019['AAAAMMJJHH'].str.startswith(('2016', '2017', '2018', '2019'))]
df_2016_2019.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,DRR1,TD,TN,...,U,UN,HUN,UX,HUX,B1,N2,B2,N3,B3
50593,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022408,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50594,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022409,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50595,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022410,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50596,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022411,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50597,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022412,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
for col in df_2016_2019.columns:
    is_col_nan = df_2016_2019[col].isna().all()

    if is_col_nan:
        df_2016_2019.drop(columns=[col], inplace=True)

df_2016_2019.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,DRR1,TD,TN,...,U,UN,HUN,UX,HUX,B1,N2,B2,N3,B3
50593,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022408,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50594,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022409,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50595,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022410,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50596,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022411,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50597,97401540,LES AVIRONS - CIRAD,-21.2395,55.3275,180,2016022412,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Fichier de données 2020 - 2022

In [8]:
data_2020_2022 = "../data/data_meteo/H_974_previous-2020-2022.csv"
df_2020_2022 = import_meteo_data(data_2020_2022)

In [9]:
df_2020_2022['AAAAMMJJHH'] = df_2020_2022['AAAAMMJJHH'].astype(str)
df_2020_2022.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,DRR1,TD,TN,...,U,UN,HUN,UX,HUX,B1,N2,B2,N3,B3
0,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091408,2.6,NaN,NaN,13.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091409,2.6,NaN,NaN,13.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091410,0.6,NaN,NaN,13.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091411,0.2,NaN,NaN,13.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091412,0.0,NaN,NaN,13.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for col in df_2020_2022.columns:
    is_col_nan = df_2020_2022[col].isna().all()

    if is_col_nan:
        df_2020_2022.drop(columns=[col], inplace=True)

df_2020_2022.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,DRR1,TD,TN,...,U,UN,HUN,UX,HUX,B1,N2,B2,N3,B3
0,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091408,2.6,NaN,NaN,13.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091409,2.6,NaN,NaN,13.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091410,0.6,NaN,NaN,13.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091411,0.2,NaN,NaN,13.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,97401520,LE TEVELAVE,-21.211667,55.361333,908,2020091412,0.0,NaN,NaN,13.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Fichier de données 2023 - 2024

In [11]:
data_2023_2024 = "../data/data_meteo/H_974_latest-2023-2024.csv"
df_2023_2024 = import_meteo_data(data_2023_2024)

In [12]:
df_2023_2024['AAAAMMJJHH'] = df_2023_2024['AAAAMMJJHH'].astype(str)
df_2023 = df_2023_2024[df_2023_2024['AAAAMMJJHH'].str.startswith(('2023'))]
df_2023

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,DRR1,TD,TN,...,U,UN,HUN,UX,HUX,B1,N2,B2,N3,B3
0,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010100,0.0,NaN,NaN,15.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010101,0.0,NaN,NaN,14.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010102,0.0,NaN,NaN,14.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010103,0.0,NaN,NaN,15.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010104,0.0,NaN,NaN,14.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703454,97424460,PALMISTE-ROUGE,-21.168667,55.474500,830,2023123119,0.0,NaN,NaN,22.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703455,97424460,PALMISTE-ROUGE,-21.168667,55.474500,830,2023123120,0.0,NaN,NaN,21.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703456,97424460,PALMISTE-ROUGE,-21.168667,55.474500,830,2023123121,0.0,NaN,NaN,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703457,97424460,PALMISTE-ROUGE,-21.168667,55.474500,830,2023123122,0.0,NaN,NaN,21.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_2023["LAT"].dtype

dtype('float64')

In [14]:
for col in df_2023.columns:
    is_col_nan = df_2023[col].isna().all()

    if is_col_nan:
        df_2023.drop(columns=[col], inplace=True)

df_2023.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,DRR1,TD,TN,...,U,UN,HUN,UX,HUX,B1,N2,B2,N3,B3
0,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010100,0.0,NaN,NaN,15.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010101,0.0,NaN,NaN,14.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010102,0.0,NaN,NaN,14.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010103,0.0,NaN,NaN,15.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,97401520,LE TEVELAVE,-21.211667,55.361333,908,2023010104,0.0,NaN,NaN,14.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Concatenation des données des 3 périodes : 2016-2019, 2020-2022, 2023

Dans cette partie nous allons concatener les dataframes df_2016_2019, df_2020_2022 et df_2023 pour avoir un seul dataframe df_2016_2023

In [15]:
data_2016_2023 = [df_2016_2019, df_2020_2022, df_2023]
df_meteo_2016_2023 = pd.concat(data_2016_2023, ignore_index=True)
df_meteo_2016_2023

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,DRR1,TD,TN,...,U,UN,HUN,UX,HUX,B1,N2,B2,N3,B3
0,97401540,LES AVIRONS - CIRAD,-21.239500,55.3275,180,2016022408,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97401540,LES AVIRONS - CIRAD,-21.239500,55.3275,180,2016022409,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97401540,LES AVIRONS - CIRAD,-21.239500,55.3275,180,2016022410,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,97401540,LES AVIRONS - CIRAD,-21.239500,55.3275,180,2016022411,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,97401540,LES AVIRONS - CIRAD,-21.239500,55.3275,180,2016022412,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4645468,97424460,PALMISTE-ROUGE,-21.168667,55.4745,830,2023123119,0.0,NaN,NaN,22.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4645469,97424460,PALMISTE-ROUGE,-21.168667,55.4745,830,2023123120,0.0,NaN,NaN,21.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4645470,97424460,PALMISTE-ROUGE,-21.168667,55.4745,830,2023123121,0.0,NaN,NaN,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4645471,97424460,PALMISTE-ROUGE,-21.168667,55.4745,830,2023123122,0.0,NaN,NaN,21.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
missing_values = df_meteo_2016_2023.isna().sum()
print(missing_values)

NUM_POSTE           0
NOM_USUEL           0
LAT                 0
LON                 0
ALTI                0
AAAAMMJJHH          0
RR1             65143
DRR1          4510128
TD            3502742
TN            1387261
HTN           1393975
TX            1387333
HTX           1394036
U             3501784
UN            3502462
HUN           3503274
UX            3502440
HUX           3503107
B1            4509738
N2            4598179
B2            4598179
N3            4622884
B3            4622884
dtype: int64


## Jointure des données des capteurs TESSW et des données météo

#### Matcher les couples de coordonnées

In [17]:
def evaluate(row):
    global compare_df
    # Given couple (x0, y0)
    x0 = row[0]
    y0 = row[1]

    # Calculate distance between each couple in the DataFrame and the given couple
    compare_df['distance'] = np.sqrt((compare_df['LAT'] - x0)**2 + (compare_df['LON'] - y0)**2)

    # Find the couple with the smallest distance
    closest_couple = compare_df.loc[compare_df['distance'].idxmin()]

    compare_df.drop(columns=['distance'])
    
    # Find the couple with the smallest distance
    return (x0,y0,closest_couple['LAT'],closest_couple['LON'])

In [18]:
compare_df = (df_meteo_2016_2023[['LAT','LON']]).drop_duplicates()


df = df_sensors[['latitude','longitude']]
df = df.drop_duplicates()

# Initialize a list to store the closest couples for each couple in df
closest_couples = []

# Iterate over each couple in df2
for index, row in df.iterrows():
    closest_couple = evaluate(row)
    closest_couples.append(closest_couple)

# Convert the list of closest couples to a DataFrame
df_coord_couple = pd.DataFrame(closest_couples, columns=['latitude', 'longitude', 'LAT', 'LON'])
df_coord_couple

C:\Users\fabri\AppData\Local\Temp\ipykernel_22780\702563028.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x0 = row[0]
C:\Users\fabri\AppData\Local\Temp\ipykernel_22780\702563028.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y0 = row[1]
C:\Users\fabri\AppData\Local\Temp\ipykernel_22780\702563028.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x0 = row[0]
C:\Users\fabri\AppData\Local\Temp\ipykernel_22780\702563028.py:5

,latitude,longitude,LAT,LON
0,-21.35870,55.66335,-21.337500,55.667667
1,-21.09551,55.48023,-21.134167,55.471667
2,-21.11493,55.72708,-21.120167,55.758667
3,-21.21602,55.31417,-21.215167,55.325833
4,-21.33032,55.56722,-21.321167,55.572333
5,-21.15302,55.43652,-21.153500,55.438167
6,-21.27726,55.51057,-21.251667,55.530333
7,-21.26694,55.49988,-21.274500,55.475333
8,-21.13665,55.62397,-21.136167,55.627167
9,-21.22228,55.34378,-21.215167,55.325833


#### Ajouter les colonnes des coordonnées météo au dataset de capteur

In [19]:
df_sensors

,UTCDataTime,LocalDateTime,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,latitude,longitude,sensor
0,2023-11-02T12:57:05.000,2023-11-02T12:57:05.000,31.1,21.7,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
1,2023-11-02T12:58:05.000,2023-11-02T12:58:05.000,31.2,21.7,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
2,2023-11-02T12:59:05.000,2023-11-02T12:59:05.000,31.5,21.8,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
3,2023-11-02T13:18:05.000,2023-11-02T13:18:05.000,28.9,20.8,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
4,2023-11-02T13:19:05.000,2023-11-02T13:19:05.000,28.7,20.4,50000.0,0.0,2.0,-21.3587,55.66335,stars1022
...,...,...,...,...,...,...,...,...,...,...
19156,2023-11-18T05:07:10.000,2023-11-18T09:07:10.000,38.2,5.1,50000.0,0.0,2.0,-20.9130,55.47980,stars975
19157,2023-11-18T05:08:13.000,2023-11-18T09:08:13.000,38.8,5.5,50000.0,0.0,2.0,-20.9130,55.47980,stars975
19158,2023-11-18T05:11:26.000,2023-11-18T09:11:26.000,36.4,0.1,50000.0,0.0,2.0,-20.9130,55.47980,stars975
19159,2023-11-18T05:12:27.000,2023-11-18T09:12:27.000,35.6,1.4,50000.0,0.0,2.0,-20.9130,55.47980,stars975


In [20]:
# Fusionner les informations de correspondance avec df_sensors
df_sensors_lat_lon = df_sensors.merge(df_coord_couple, on=['latitude', 'longitude'], how='left')

# Afficher le DataFrame mis à jour
df_sensors_lat_lon

,UTCDataTime,LocalDateTime,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,latitude,longitude,sensor,LAT,LON
0,2023-11-02T12:57:05.000,2023-11-02T12:57:05.000,31.1,21.7,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667
1,2023-11-02T12:58:05.000,2023-11-02T12:58:05.000,31.2,21.7,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667
2,2023-11-02T12:59:05.000,2023-11-02T12:59:05.000,31.5,21.8,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667
3,2023-11-02T13:18:05.000,2023-11-02T13:18:05.000,28.9,20.8,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667
4,2023-11-02T13:19:05.000,2023-11-02T13:19:05.000,28.7,20.4,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667
...,...,...,...,...,...,...,...,...,...,...,...,...
3862203,2023-11-18T05:07:10.000,2023-11-18T09:07:10.000,38.2,5.1,50000.0,0.0,2.0,-20.9130,55.47980,stars975,-20.8970,55.495000
3862204,2023-11-18T05:08:13.000,2023-11-18T09:08:13.000,38.8,5.5,50000.0,0.0,2.0,-20.9130,55.47980,stars975,-20.8970,55.495000
3862205,2023-11-18T05:11:26.000,2023-11-18T09:11:26.000,36.4,0.1,50000.0,0.0,2.0,-20.9130,55.47980,stars975,-20.8970,55.495000
3862206,2023-11-18T05:12:27.000,2023-11-18T09:12:27.000,35.6,1.4,50000.0,0.0,2.0,-20.9130,55.47980,stars975,-20.8970,55.495000


#### Faire la jointure entre les deux datasets

In [21]:
# Convertir la colonne "LocalDateTime" en format datetime
df_sensors_lat_lon['LocalDateTime'] = pd.to_datetime(df_sensors_lat_lon['LocalDateTime'])

# Formater la date selon le format AAAAMMJJHH
df_sensors_lat_lon['AAAAMMJJHH'] = df_sensors_lat_lon['LocalDateTime'].dt.strftime('%Y%m%d%H')

# Afficher le DataFrame avec la nouvelle colonne
df_sensors_lat_lon.head()

,UTCDataTime,LocalDateTime,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,latitude,longitude,sensor,LAT,LON,AAAAMMJJHH
0,2023-11-02T12:57:05.000,2023-11-02 12:57:05,31.1,21.7,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667,2023110212
1,2023-11-02T12:58:05.000,2023-11-02 12:58:05,31.2,21.7,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667,2023110212
2,2023-11-02T12:59:05.000,2023-11-02 12:59:05,31.5,21.8,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667,2023110212
3,2023-11-02T13:18:05.000,2023-11-02 13:18:05,28.9,20.8,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667,2023110213
4,2023-11-02T13:19:05.000,2023-11-02 13:19:05,28.7,20.4,50000.0,0.0,2.0,-21.3587,55.66335,stars1022,-21.3375,55.667667,2023110213


In [22]:
# Liste des colonnes numériques à moyenner
columns_to_mean = ['EnclosureTemp', 'SkyTemp', 'Frequency', 'MSAS', 'ZP']

# # Spécifier les fonctions d'agrégation pour chaque colonne
# aggregation_functions = {'EnclosureTemp': 'mean',
#                         'SkyTemp': 'mean',
#                         'Frequency': 'mean',
#                         'MSAS': 'mean',
#                         'ZP': 'mean',}

aggregation_functions = {'EnclosureTemp': 'mean',
                        'SkyTemp': 'mean',
                        'Frequency': 'mean',
                        'MSAS': 'mean',
                        'ZP': 'mean',
                        'UTCDataTime': 'first',  # Par exemple, pour les autres colonnes
                        'LocalDateTime': 'first',
                        'latitude': 'first',
                        'longitude': 'first',
                        'LAT': 'first',
                        'LON': 'first'}

# Grouper les données par les colonnes "sensor" et "LocalAAAAMMJJHH" et appliquer les fonctions d'agrégation
df_sensors_grouped = df_sensors_lat_lon.groupby(['sensor', 'AAAAMMJJHH']).agg(aggregation_functions).reset_index()

# df_sensors_grouped = df_sensors_lat_lon.groupby(['sensor', 'LocalAAAAMMJJHH'])[columns_to_mean].mean().reset_index()


df_sensors_grouped


,sensor,AAAAMMJJHH,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,UTCDataTime,LocalDateTime,latitude,longitude,LAT,LON
0,stars1022,2023110212,31.266667,21.733333,50000.000000,0.000000,2.0,2023-11-02T12:57:05.000,2023-11-02 12:57:05,-21.3587,55.66335,-21.3375,55.667667
1,stars1022,2023110213,28.955556,21.500000,45766.834444,1.301111,2.0,2023-11-02T13:18:05.000,2023-11-02 13:18:05,-21.3587,55.66335,-21.3375,55.667667
2,stars1022,2023110214,25.281481,12.181481,16112.218148,10.080741,2.0,2023-11-02T14:33:18.000,2023-11-02 14:33:18,-21.3587,55.66335,-21.3375,55.667667
3,stars1022,2023110215,23.928333,5.470000,11.338833,19.356000,2.0,2023-11-02T15:00:21.000,2023-11-02 15:00:21,-21.3587,55.66335,-21.3375,55.667667
4,stars1022,2023110216,23.275000,5.363333,0.906333,20.513833,2.0,2023-11-02T16:00:27.000,2023-11-02 16:00:27,-21.3587,55.66335,-21.3375,55.667667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78599,stars975,2023111722,24.666667,8.200000,15.413333,18.226667,2.0,2023-11-17T18:00:03.000,2023-11-17 22:00:03,-20.9130,55.47980,-20.8970,55.495000
78600,stars975,2023111806,24.937500,5.025000,50000.000000,0.000000,2.0,2023-11-18T02:07:19.000,2023-11-18 06:07:19,-20.9130,55.47980,-20.8970,55.495000
78601,stars975,2023111807,33.590909,2.018182,50000.000000,0.000000,2.0,2023-11-18T03:09:40.000,2023-11-18 07:09:40,-20.9130,55.47980,-20.8970,55.495000
78602,stars975,2023111808,37.858333,5.508333,50000.000000,0.000000,2.0,2023-11-18T04:16:17.000,2023-11-18 08:16:17,-20.9130,55.47980,-20.8970,55.495000


In [23]:
douzeHeure = df_sensors_grouped[df_sensors_grouped['AAAAMMJJHH'] == "2023110212"]
douzeHeure

,sensor,AAAAMMJJHH,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,UTCDataTime,LocalDateTime,latitude,longitude,LAT,LON
0,stars1022,2023110212,31.266667,21.733333,50000.000000,0.000000,2.00,2023-11-02T12:57:05.000,2023-11-02 12:57:05,-21.35870,55.66335,-21.337500,55.667667
34132,stars712,2023110212,28.633333,11.658333,50000.000000,0.000000,20.50,2023-11-02T08:19:43.000,2023-11-02 12:19:43,-21.13665,55.62397,-21.136167,55.627167
38477,stars713,2023110212,28.866667,25.700000,50000.000000,0.000000,20.30,2023-11-02T08:01:46.000,2023-11-02 12:01:46,-21.22228,55.34378,-21.215167,55.325833
43512,stars737,2023110212,15.600000,16.700000,50000.000000,0.000000,20.32,2023-11-02T08:11:18.000,2023-11-02 12:11:18,-21.07937,55.38324,-21.076667,55.381167
51008,stars874,2023110212,42.566667,22.316667,50000.000000,0.000000,2.00,2023-11-02T08:01:44.000,2023-11-02 12:01:44,-20.99251,55.28307,-21.027000,55.268333
61582,stars955,2023110212,40.133333,20.983333,50000.000000,0.000000,20.31,2023-11-02T08:42:44.000,2023-11-02 12:42:44,-20.93046,55.65197,-20.934500,55.664500
63712,stars959,2023110212,45.446667,16.113333,49846.166667,0.051333,2.00,2023-11-02T08:39:21.000,2023-11-02 12:39:21,-21.27472,55.44096,-21.245833,55.428000
67278,stars961,2023110212,32.075000,25.775000,50000.000000,0.000000,2.00,2023-11-02T08:01:36.000,2023-11-02 12:01:36,-21.25474,55.36288,-21.265000,55.380000
67538,stars962,2023110212,28.575000,20.328571,50000.000000,0.000000,2.00,2023-11-02T12:00:20.000,2023-11-02 12:00:20,-21.08778,55.22990,-21.105167,55.247667
71107,stars964,2023110212,30.100000,21.562500,50000.000000,0.000000,2.00,2023-11-02T08:00:04.000,2023-11-02 12:00:04,-21.36291,55.59633,-21.385167,55.609667


In [24]:
missing_values = df_sensors_grouped.isna().sum()
print(missing_values)

sensor           0
AAAAMMJJHH       0
EnclosureTemp    0
SkyTemp          0
Frequency        0
MSAS             0
ZP               0
UTCDataTime      0
LocalDateTime    0
latitude         0
longitude        0
LAT              0
LON              0
dtype: int64


In [43]:
# Fusionner les informations de df_sensors_grouped avec df_meteo_2016_2023
df_final = df_sensors_grouped.merge(df_meteo_2016_2023, on=['LAT', 'LON', 'AAAAMMJJHH'], how='inner')

# Afficher le DataFrame mis à jour
df_final

,sensor,AAAAMMJJHH,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,UTCDataTime,LocalDateTime,latitude,...,U,UN,HUN,UX,HUX,B1,N2,B2,N3,B3
0,stars1022,2023110212,31.266667,21.733333,50000.000000,0.000000,2.0,2023-11-02T12:57:05.000,2023-11-02 12:57:05,-21.3587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,stars1022,2023110213,28.955556,21.500000,45766.834444,1.301111,2.0,2023-11-02T13:18:05.000,2023-11-02 13:18:05,-21.3587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,stars1022,2023110214,25.281481,12.181481,16112.218148,10.080741,2.0,2023-11-02T14:33:18.000,2023-11-02 14:33:18,-21.3587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,stars1022,2023110215,23.928333,5.470000,11.338833,19.356000,2.0,2023-11-02T15:00:21.000,2023-11-02 15:00:21,-21.3587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,stars1022,2023110216,23.275000,5.363333,0.906333,20.513833,2.0,2023-11-02T16:00:27.000,2023-11-02 16:00:27,-21.3587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69431,stars975,2023111722,24.666667,8.200000,15.413333,18.226667,2.0,2023-11-17T18:00:03.000,2023-11-17 22:00:03,-20.9130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69432,stars975,2023111806,24.937500,5.025000,50000.000000,0.000000,2.0,2023-11-18T02:07:19.000,2023-11-18 06:07:19,-20.9130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69433,stars975,2023111807,33.590909,2.018182,50000.000000,0.000000,2.0,2023-11-18T03:09:40.000,2023-11-18 07:09:40,-20.9130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69434,stars975,2023111808,37.858333,5.508333,50000.000000,0.000000,2.0,2023-11-18T04:16:17.000,2023-11-18 08:16:17,-20.9130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
missing_values = df_final.isna().sum()
print(missing_values)

sensor               0
AAAAMMJJHH           0
EnclosureTemp        0
SkyTemp              0
Frequency            0
MSAS                 0
ZP                   0
UTCDataTime          0
LocalDateTime        0
latitude             0
longitude            0
LAT                  0
LON                  0
NUM_POSTE            0
NOM_USUEL            0
ALTI                 0
RR1               1009
DRR1             69436
TD               47975
TN                8221
HTN               8461
TX                8222
HTX               8462
U                47975
UN               47975
HUN              47975
UX               47975
HUX              47975
B1               69436
N2               69436
B2               69436
N3               69436
B3               69436
dtype: int64


In [45]:
for col in df_final.columns:
    is_col_nan = df_final[col].isna().all()

    if is_col_nan:
        df_final.drop(columns=[col], inplace=True)

df_final

,sensor,AAAAMMJJHH,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,UTCDataTime,LocalDateTime,latitude,...,TD,TN,HTN,TX,HTX,U,UN,HUN,UX,HUX
0,stars1022,2023110212,31.266667,21.733333,50000.000000,0.000000,2.0,2023-11-02T12:57:05.000,2023-11-02 12:57:05,-21.3587,...,NaN,24.0,1157.0,24.9,1101.0,NaN,NaN,NaN,NaN,NaN
1,stars1022,2023110213,28.955556,21.500000,45766.834444,1.301111,2.0,2023-11-02T13:18:05.000,2023-11-02 13:18:05,-21.3587,...,NaN,24.2,1201.0,26.4,1259.0,NaN,NaN,NaN,NaN,NaN
2,stars1022,2023110214,25.281481,12.181481,16112.218148,10.080741,2.0,2023-11-02T14:33:18.000,2023-11-02 14:33:18,-21.3587,...,NaN,24.8,1358.0,26.9,1302.0,NaN,NaN,NaN,NaN,NaN
3,stars1022,2023110215,23.928333,5.470000,11.338833,19.356000,2.0,2023-11-02T15:00:21.000,2023-11-02 15:00:21,-21.3587,...,NaN,23.1,1450.0,25.3,1403.0,NaN,NaN,NaN,NaN,NaN
4,stars1022,2023110216,23.275000,5.363333,0.906333,20.513833,2.0,2023-11-02T16:00:27.000,2023-11-02 16:00:27,-21.3587,...,NaN,22.5,1559.0,23.6,1501.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69431,stars975,2023111722,24.666667,8.200000,15.413333,18.226667,2.0,2023-11-17T18:00:03.000,2023-11-17 22:00:03,-20.9130,...,NaN,24.8,2143.0,25.1,2101.0,NaN,NaN,NaN,NaN,NaN
69432,stars975,2023111806,24.937500,5.025000,50000.000000,0.000000,2.0,2023-11-18T02:07:19.000,2023-11-18 06:07:19,-20.9130,...,NaN,22.1,555.0,22.3,501.0,NaN,NaN,NaN,NaN,NaN
69433,stars975,2023111807,33.590909,2.018182,50000.000000,0.000000,2.0,2023-11-18T03:09:40.000,2023-11-18 07:09:40,-20.9130,...,NaN,22.2,601.0,23.6,700.0,NaN,NaN,NaN,NaN,NaN
69434,stars975,2023111808,37.858333,5.508333,50000.000000,0.000000,2.0,2023-11-18T04:16:17.000,2023-11-18 08:16:17,-20.9130,...,NaN,23.6,701.0,26.0,759.0,NaN,NaN,NaN,NaN,NaN


In [46]:
missing_values = df_final.isna().sum()
print(missing_values)

sensor               0
AAAAMMJJHH           0
EnclosureTemp        0
SkyTemp              0
Frequency            0
MSAS                 0
ZP                   0
UTCDataTime          0
LocalDateTime        0
latitude             0
longitude            0
LAT                  0
LON                  0
NUM_POSTE            0
NOM_USUEL            0
ALTI                 0
RR1               1009
TD               47975
TN                8221
HTN               8461
TX                8222
HTX               8462
U                47975
UN               47975
HUN              47975
UX               47975
HUX              47975
dtype: int64


In [47]:
columns_to_drop = ["LAT", "LON", "UTCDataTime"]
df_final.drop(columns=columns_to_drop, inplace=True)
df_final

,sensor,AAAAMMJJHH,EnclosureTemp,SkyTemp,Frequency,MSAS,ZP,LocalDateTime,latitude,longitude,...,TD,TN,HTN,TX,HTX,U,UN,HUN,UX,HUX
0,stars1022,2023110212,31.266667,21.733333,50000.000000,0.000000,2.0,2023-11-02 12:57:05,-21.3587,55.66335,...,NaN,24.0,1157.0,24.9,1101.0,NaN,NaN,NaN,NaN,NaN
1,stars1022,2023110213,28.955556,21.500000,45766.834444,1.301111,2.0,2023-11-02 13:18:05,-21.3587,55.66335,...,NaN,24.2,1201.0,26.4,1259.0,NaN,NaN,NaN,NaN,NaN
2,stars1022,2023110214,25.281481,12.181481,16112.218148,10.080741,2.0,2023-11-02 14:33:18,-21.3587,55.66335,...,NaN,24.8,1358.0,26.9,1302.0,NaN,NaN,NaN,NaN,NaN
3,stars1022,2023110215,23.928333,5.470000,11.338833,19.356000,2.0,2023-11-02 15:00:21,-21.3587,55.66335,...,NaN,23.1,1450.0,25.3,1403.0,NaN,NaN,NaN,NaN,NaN
4,stars1022,2023110216,23.275000,5.363333,0.906333,20.513833,2.0,2023-11-02 16:00:27,-21.3587,55.66335,...,NaN,22.5,1559.0,23.6,1501.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69431,stars975,2023111722,24.666667,8.200000,15.413333,18.226667,2.0,2023-11-17 22:00:03,-20.9130,55.47980,...,NaN,24.8,2143.0,25.1,2101.0,NaN,NaN,NaN,NaN,NaN
69432,stars975,2023111806,24.937500,5.025000,50000.000000,0.000000,2.0,2023-11-18 06:07:19,-20.9130,55.47980,...,NaN,22.1,555.0,22.3,501.0,NaN,NaN,NaN,NaN,NaN
69433,stars975,2023111807,33.590909,2.018182,50000.000000,0.000000,2.0,2023-11-18 07:09:40,-20.9130,55.47980,...,NaN,22.2,601.0,23.6,700.0,NaN,NaN,NaN,NaN,NaN
69434,stars975,2023111808,37.858333,5.508333,50000.000000,0.000000,2.0,2023-11-18 08:16:17,-20.9130,55.47980,...,NaN,23.6,701.0,26.0,759.0,NaN,NaN,NaN,NaN,NaN


In [48]:
df_final.to_csv('../data/data_tessw_meteo.csv', index=False)